In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# Load data
file_path = 'data\Seamen Report.xlsx'
#data = pd.read_excel(file_path, sheet_name='Container-Deck')
data = pd.read_excel(file_path, sheet_name='Container-Engine')
#data = pd.read_excel(file_path, sheet_name='manalagi-Deck')
#data = pd.read_excel(file_path, sheet_name='manalag-Engine')

data.head(10)

,SEAMAN CODE,SEAMAN NAME,RANK,VESSEL,VESSEL GROUP ID,BAGIAN,EDUCATION,SEAFARER CODE,UMUR,PHONE1,PHONE2,PHONE3,PHONE4,CERTIFICATE
0,20040159,HERMAN BUMBUNGAN,NAKHODA,KM. ORIENTAL EMERALD,1E,Engine,PIP MAKASAR,6200016260,52,04114661062,81217260616,81314123353,-,ANT-I
1,20040002,MAD KAINI,MUALIM I,KM. ORIENTAL EMERALD,1E,Engine,AKPELNI,6200521998,53,081361730508,81361730508,-,-,ANT-II
2,20160111,PANGURIPAN GAJA MADA SITANGGAN,MUALIM II,KM. ORIENTAL EMERALD,1E,Engine,AMI MEDAN,6201407512,31,0214711091,82334341117,82226058992,-,ANT-III
3,20190357,ALIP NUR FATCHURAHMAN,MUALIM III,KM. ORIENTAL EMERALD,1E,Engine,Poktepel Surabaya,6211721761,27,081326394425,81326394425,-,-,ANT-III
4,20040482,EDY SUBIYANTORO,KKM,KM. ORIENTAL EMERALD,1E,Engine,AMNI SEMARANAG,6200060301,52,0342552221,81234321072,-,-,ATT-I
5,20120710,VINKY ARIJONA SIMANJUNTAK,MASINIS I,KM. ORIENTAL EMERALD,1E,Engine,AKPELNI SEMARANG,6201592974,37,081392949048,-,-,-,ATT-II
6,20170427,PUGUH APRIL RIYANTO,MASINIS II,KM. ORIENTAL EMERALD,1E,Engine,PIP Semarang,6211705009,27,081232512660,-,-,-,ATT-II
7,20200213,THAUFIQ WICAKSONO,MASINIS III,KM. ORIENTAL EMERALD,1E,Engine,POLTEKPEL,6211919528,25,085777788077,8563072189,-,-,ATT-III
8,20060128,CHRISTIAN PALIT,SERANG,KM. ORIENTAL EMERALD,1E,Engine,SMK TOLI TOLI,6200196188,52,0218861820,81382734823,-,-,ANT-D
9,20240005,ADE YOGA PRADANA,JURU MUDI,KM. ORIENTAL EMERALD,1E,Engine,SPM Bakti Samudera,6211536790,27,082233210401,-,-,-,ANT-IV


In [5]:
def filter_data(df, bagian, vessel_group_id):
    return df[(df['BAGIAN'] == bagian) & (df['VESSEL GROUP ID'] == vessel_group_id)]

def calculate_similarity(df, rank, certificate, age_range):
    df = df[(df['UMUR'] >= age_range[0]) & (df['UMUR'] <= age_range[1])]
    df['combined_features'] = df['RANK'] + " " + df['CERTIFICATE']

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['combined_features'])

    user_input = [rank + " " + certificate]
    user_vector = vectorizer.transform(user_input)

    similarity_scores = cosine_similarity(user_vector, tfidf_matrix).flatten()

    df['similarity_score'] = similarity_scores
    df = df.sort_values(by='similarity_score', ascending=False)
    
    return df[['SEAMAN NAME', 'SEAMAN CODE', 'SEAFARER CODE', 'RANK', 'VESSEL', 'UMUR', 'CERTIFICATE', 'similarity_score']]


In [6]:
# usage example
bagian = 'Engine'
vessel = 'KM. ORIENTAL EMERALD'
vessel_group_id = data[data['VESSEL'] == vessel]['VESSEL GROUP ID'].values[0]
rank = 'NAKHODA'
certificate = 'ANT-I'
age_range = (45, 65)


filtered_data = filter_data(data, bagian, vessel_group_id)
result = calculate_similarity(filtered_data, rank, certificate, age_range)

C:\Users\ilham\AppData\Local\Temp\ipykernel_17068\4214825147.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combined_features'] = df['RANK'] + " " + df['CERTIFICATE']
C:\Users\ilham\AppData\Local\Temp\ipykernel_17068\4214825147.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['similarity_score'] = similarity_scores


In [9]:
result.head()

,SEAMAN NAME,SEAMAN CODE,SEAFARER CODE,RANK,VESSEL,UMUR,CERTIFICATE,similarity_score
0,HERMAN BUMBUNGAN,20040159,6200016260,NAKHODA,KM. ORIENTAL EMERALD,52,ANT-I,1.0
61,HADI SUSANTO,20140333,6201392247,NAKHODA,KM. ORIENTAL GOLD,64,ANT-I,1.0
167,SYAHRUDDIN WAHAB,20160306,6200299350,NAKHODA,KM. LUZON,60,ANT-I,1.0
19,MARSAID TEGUH SANTOSO,20180263,6200083213,NAKHODA,KM. ORIENTAL RUBY,58,ANT-I,1.0
147,BOBBY BLESSY PANGALILA,20170526,6200060469,NAKHODA,KM. VERIZON,47,ANT-I,1.0
